### Data Cleaning

Tasks:
- convert datetime to actual dt object with this notation `%YYYY-%MM-%DD %HH:%mm:%ss` to ensure max compatibility with existing code
- clean comments all the `,` and `[ ]` need to be removed, the string needs to be wrapped in quotation marks to avoid breaking the csv
- create a country field, for cities in the us it's fairly easy: filter every field in 'state' that does not have `<br/>` as value. For cities outside it's a bit more complicated, maybe it's possible to use the 'city' field and get the string between parenthesis (which usually contains info on countries.
- add coordinates to cities (for now feasible solution is: https://geopy.readthedocs.io/en/stable/#

In [376]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ufo")

In [2]:
allYears = pd.read_csv('data/allyears.csv')

In [3]:
allYears.head(5)

,Unnamed: 0,datetime,city,state,shape,comment
0,0,12/23/20 07:30,Canadian Lakes,MI,Light,"light in sky ,no sound"
1,1,12/23/20 03:18,Grants,CA,Light,Bright Orb seen below plane level at Mach 1+ S...
2,2,12/23/20 03:00,East Quogue,NY,Other,"flickering colored lights in the night sky, in..."
3,3,12/22/20 22:45,Indian River Shores,FL,Changing,Orange orb over Indian River Shores Florida
4,4,12/22/20 22:05,lake elsinore,CA,Light,saw a glowing object.shape not discerned. trav...


Dropping accidentally scraped duplicates

In [224]:
noDuplicates = allYears.drop_duplicates(subset="comment", keep="first")

In [225]:
consistentStructure = noDuplicates.drop(['Unnamed: 0'], axis=1)

In [226]:
#check nan
noDuplicates.isna().sum()

Unnamed: 0    0
datetime      0
city          0
state         0
shape         0
comment       1
dtype: int64

Convert datetime from string to timedate obj

In [227]:
consistentStructure['datetime'] = pd.to_datetime(consistentStructure['datetime'])

In [228]:
consistentStructure.head(5)

,datetime,city,state,shape,comment
0,2020-12-23 07:30:00,Canadian Lakes,MI,Light,"light in sky ,no sound"
1,2020-12-23 03:18:00,Grants,CA,Light,Bright Orb seen below plane level at Mach 1+ S...
2,2020-12-23 03:00:00,East Quogue,NY,Other,"flickering colored lights in the night sky, in..."
3,2020-12-22 22:45:00,Indian River Shores,FL,Changing,Orange orb over Indian River Shores Florida
4,2020-12-22 22:05:00,lake elsinore,CA,Light,saw a glowing object.shape not discerned. trav...


In [229]:
# check formats
consistentStructure.dtypes

datetime    datetime64[ns]
city                object
state               object
shape               object
comment             object
dtype: object

In [230]:
# Creating empty column for country
consistentStructure["country"] = ""

In [231]:
# Assigning country to US cities
consistentStructure["country"] = np.where(consistentStructure['state'] != '<br/>', 'USA', 'nan')

In [232]:
# Taking care of Canada to try out numpy method
isElsewhere = (consistentStructure['city'].str.contains('Canada')) & (consistentStructure['state'] != '<br/>')
replace = ['Canada']
consistentStructure["country"] = np.where(isElsewhere, replace, consistentStructure["country"])

In [369]:
consistentStructure.sample(5)

,datetime,city,state,shape,comment,country
8156,2019-11-13 18:10:00,Scottsdale,USA,Light,Solid bright red light (thought it was a small...,USA
16140,2018-03-02 22:47:00,Lone Jack,USA,Circle,I observed a round lighted object start overhe...,USA
14501,2018-08-18 19:40:00,Henderson,USA,Changing,"Brilliant white light, morphed to clearly defi...",USA
10328,2019-08-01 23:00:00,Wayne,USA,Other,"Saw multiple aircrafts, a bright white light a...",USA
10334,2019-08-01 22:30:00,Ridgewood,USA,Rectangle,Flat black looking car sized rectangle with a ...,USA


In [234]:
# Filtering rows with parentheses, hence country information
otherCountries = consistentStructure.loc[(consistentStructure['city'].str.contains('\(')) & (consistentStructure['state'] == '<br/>')]

In [235]:
otherCountries['city'].tolist()
OnlyCountries = []
# Looping to split strings and get unique countries value (working decently, but not perfectly)
for country in otherCountries['city']:
    splitted = country.split('(', 1)
    cleaned = splitted[1].split(')', 1)
    print(''+ cleaned[0] + '')
    OnlyCountries.append(cleaned[0])
    
uniqueCountries = list(set(OnlyCountries))

Japan
Ireland
UK/England
UK/England
Argentina
Tasmania
India
UK/Scotland
Mexico
UK/England
Mexico
Macedonia
Brazil
Russia
India
Bahamas
UK/England
Ireland
Israel
Ecuador
Switzerland
Indonesia
Norway
UK/England
UK/England
Australia
Indonesia
Germany
Wilhelmsburg
Indonesia
Australia
Germany
UK/England
Indonesia
Brazil
Czech Republic
Australia
India
UK/England
Australia
UK/England
Punjab
Norway
Nunavut
Brazil
Indonesia
UK/England
Ireland
UK/England
Switzerland
UK/England
Thailand
México
UK/England
UK/England
UK/Scotland
UK/England
Tawau
Germany
India
Rondonia
Northern Cyprus
UK/England
Germany
UK/England
Western Australia
India
Australia
India
Russia
Iran
Armenia
Brazil
Mexico
Australia
Philippines
India
Chile
Germany
UK/England
India
UK/England
India
Croatia
Jordan
Hungary
India
India
from the ISS
Brazil
Croatia
Germany
Argentina
Queensland
India
New Zealand
India
Thailand
France
Kenya
UK/England
Germany
Australia
Puerto Rico
Seychelles
France
Switzerland
UK/England
Village of
India
Indi

In [236]:
otherCountries.sample(10)

,datetime,city,state,shape,comment,country
4837,2020-04-09 17:20:00,Tewantin (Queensland)(Australia),<br/>,Light,15to25lights very high altitude in one line fo...,nan
20291,2017-04-30 02:00:00,Oxford (UK/England),<br/>,Circle,8 lights going spinning in big circles. Very h...,nan
13359,2018-12-20 17:30:00,London (UK/England),<br/>,Light,Static green light dropped vertically changer ...,nan
2170,2020-08-10 21:45:00,Balnarring (Australia),<br/>,Light,Bright light. Flash twice. Moving extremely sl...,nan
5519,2020-03-08 05:00:00,London (UK/England),<br/>,<br/>,Seen moving 'stars' all moving in the same dir...,nan
15526,2018-05-20 18:18:00,Glasgow (UK/Scotland),<br/>,Triangle,Photograph of two triangular objects in the fr...,nan
1411,2020-09-19 00:26:00,Tulum (Mexico),<br/>,Circle,Fireball Cylinder Shaped Craft over Tulum spot...,nan
5665,2020-03-05 19:43:00,Raigad (India),<br/>,Disk,While sky watching my camera caught 5 mysterio...,nan
6392,2020-02-02 04:00:00,White River (South Africa),<br/>,Formation,Star like strand of lights moving south to nor...,nan
2905,2020-07-05 07:45:00,Jakarta (Indonesia),<br/>,Cigar,Object Seemed to be shrouded in a halo... Enla...,nan


In [286]:
# Looping over countries list to assign correct country to specific cities
for country in OnlyCountries:
    isElsewhere = (consistentStructure['city'].str.contains(country)) & (consistentStructure['state'] == '<br/>')
    print(country)
    consistentStructure["country"] = np.where(isElsewhere, country, consistentStructure["country"])  

Japan
Ireland
UK/England
UK/England
Argentina
Tasmania
India
UK/Scotland
Mexico
UK/England
Mexico
Macedonia
Brazil
Russia
India
Bahamas
UK/England
Ireland
Israel
Ecuador
Switzerland
Indonesia
Norway
UK/England
UK/England
Australia
Indonesia
Germany
Wilhelmsburg
Indonesia
Australia
Germany
UK/England
Indonesia
Brazil
Czech Republic
Australia
India
UK/England
Australia
UK/England
Punjab
Norway
Nunavut
Brazil
Indonesia
UK/England
Ireland
UK/England
Switzerland
UK/England
Thailand
México
UK/England
UK/England
UK/Scotland
UK/England
Tawau
Germany
India
Rondonia
Northern Cyprus
UK/England
Germany
UK/England
Western Australia
India
Australia
India
Russia
Iran
Armenia
Brazil
Mexico
Australia
Philippines
India
Chile
Germany
UK/England
India
UK/England
India
Croatia
Jordan
Hungary
India
India
from the ISS
Brazil
Croatia
Germany
Argentina
Queensland
India
New Zealand
India
Thailand
France
Kenya
UK/England
Germany
Australia
Puerto Rico
Seychelles
France
Switzerland
UK/England
Village of
India
Indi

error: nothing to repeat at position 0

In [288]:
# New data to be safe
allCountries = consistentStructure

In [332]:
# Not really nice: just checking for weird countries and manually replacing deviating values
allCountries["country"] = np.where(allCountries['country'] == 'City', 'USA', allCountries['country'])

In [282]:
allCountries.sample(50)

,datetime,city,state,shape,comment,country
16815,2017-12-23 21:00:00,Barre,VT,Flash,Multiple large muted firework type flashes in ...,USA
3445,2020-05-26 22:30:00,Mars,PA,Triangle,Hovering triangular object with 3 white lights...,USA
9756,2019-09-08 04:30:00,Hurricane,UT,Light,Red colored orb sitting high in sky in one spo...,USA
1442,2020-09-16 22:00:00,Richboro,PA,Circle,I saw ufo flying in the sky late at night,USA
4721,2020-04-13 20:00:00,Lincolnton,GA,Light,My grandparents saw about 30 ufos last night a...,USA
305,2020-11-27 03:30:00,Layton,UT,Other,Anti gravity no visible thrust and no sound it...,USA
10399,2019-07-29 12:00:00,Palm Desert,CA,Formation,V shaped formation in the sky formed by round ...,USA
17645,2017-10-27 15:15:00,Chicago,IL,Sphere,I was headed southbound on the CTA bus on Dame...,USA
16658,2018-01-05 17:20:00,Fredrick,MD,Light,One single bright light at high rate of speed,USA
213,2020-12-03 19:00:00,niwot,CO,Light,strange 10+ perfectly diagonal lights over bou...,USA


In [326]:
allCountries.loc[allCountries["country"]== 'City']

,datetime,city,state,shape,comment,country
1231,2020-10-01 06:00:00,Temple City,<br/>,Changing,Object approached from south at 400 feet in al...,City
6112,2020-02-17 18:00:00,Carson City,<br/>,Circle,Stationary bright light.every evening1800hrs t...,City
6457,2020-01-28 18:30:00,Benton City,<br/>,Other,I just got home around 630 pm sky was full of ...,City
6625,2020-01-16 17:00:00,Ocean City,<br/>,Light,Very bright stationary still light for over h...,City
19669,2017-06-18 04:00:00,Panama City (Panama),<br/>,Oval,Brightly lit almond-shaped craft static in the...,City
20431,2017-04-19 20:10:00,Chiba City (Japan),<br/>,Unknown,I saw a fast pretty round object from the othe...,City
21531,2017-01-27 22:00:00,Elk City,<br/>,Light,Singular white light in western sky . West of ...,City


In [349]:
# Removing state, too US centric
noState = allCountries.drop(['state'], axis=1)

In [350]:
# Reordering columns
reorderedData = noState[['datetime', 'city', 'country', 'shape', 'comment']]

In [355]:
# Cleaning shape column
reorderedData["shape"] = np.where(reorderedData["shape"] == '<br/>', 'nan', reorderedData["shape"])

In [358]:
# Cleaning comment column
reorderedData["comment"] = np.where(reorderedData["comment"] == '<br/>', 'nan', reorderedData["comment"])

In [370]:
reorderedData.head(5)

,datetime,city,country,shape,comment
0,2020-12-23 07:30:00,Canadian Lakes,USA,Light,"light in sky ,no sound"
1,2020-12-23 03:18:00,Grants,USA,Light,Bright Orb seen below plane level at Mach 1+ S...
2,2020-12-23 03:00:00,East Quogue,USA,Other,"flickering colored lights in the night sky, in..."
3,2020-12-22 22:45:00,Indian River Shores,USA,Changing,Orange orb over Indian River Shores Florida
4,2020-12-22 22:05:00,lake elsinore,USA,Light,saw a glowing object.shape not discerned. trav...


In [364]:
# Removing rows with MADAR node
noMADAR = reorderedData[~reorderedData["comment"].str.contains("MADAR", na=False)]

In [453]:
# Safety exporting
noMADAR.to_csv('data/sightnings_with_coords.csv', sep=",")

In [452]:
noMADAR

,datetime,city,country,shape,comment,latitude,longitude,year
0,2020-12-23 07:30:00,Canadian Lakes,USA,Light,"light in sky ,no sound",43.587070,-85.298159,2020
1,2020-12-23 03:18:00,Grants,USA,Light,Bright Orb seen below plane level at Mach 1+ S...,35.147260,-107.851446,2020
2,2020-12-23 03:00:00,East Quogue,USA,Other,"flickering colored lights in the night sky, in...",40.851465,-72.577922,2020
3,2020-12-22 22:45:00,Indian River Shores,USA,Changing,Orange orb over Indian River Shores Florida,27.716696,-80.384217,2020
4,2020-12-22 22:05:00,lake elsinore,USA,Light,saw a glowing object.shape not discerned. trav...,33.668077,-117.327262,2020
...,...,...,...,...,...,...,...,...
21810,2017-01-01 00:07:00,Jacksonville,USA,Light,A total of 5 orange-red spheres/lights spotted...,30.332184,-81.655651,2017
21811,2017-01-01 00:05:00,Rancho Cordova,USA,Egg,I seen 5 reddish orange objects flying over my...,38.589072,-121.302728,2017
21812,2017-01-01 00:01:00,San Jose,USA,Triangle,Midnight struck on new years and I walked outs...,37.336191,-121.890583,2017
21813,2017-01-01 00:00:00,Lahore (Pakistan),Pakistan,Cone,Many orange orbs,31.565682,74.314183,2017


In [426]:
from geopy.extra.rate_limiter import RateLimiter
dataframeLength = len(noMADAR.index)
elementPosition = 0
for city in allCities:
    locator = Nominatim(user_agent="UfoLoc")
    print(city)
    # 1 - conveneint function to delay between geocoding calls
    geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
    location = geocode(city)
    elementPosition = elementPosition + 1
    # 2- - create location column
    if location != None:
        isCity = noMADAR['city'] == city
        noMADAR["latitude"] = np.where(isCity, location.latitude, noMADAR["latitude"])
        noMADAR["longitude"] = np.where(isCity, location.longitude, noMADAR["longitude"])
        print(str(elementPosition) + ' / ' + str(dataframeLength))
        print(str(location.latitude) + ', ' + str(location.longitude))
    else:
        noMADAR["latitude"] = np.where(isCity, 'nan', noMADAR["latitude"])
        noMADAR["longitude"] = np.where(isCity, 'nan', noMADAR["longitude"])
        print(str(elementPosition) + ' /'  + str(dataframeLength))
    

Canadian Lakes
1 / 20136
43.58707, -85.29815868925814
Grants


<ipython-input-426-e8d1709e9bfb>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["latitude"] = np.where(isCity, location.latitude, noMADAR["latitude"])
<ipython-input-426-e8d1709e9bfb>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["longitude"] = np.where(isCity, location.longitude, noMADAR["longitude"])


2 / 20136
35.14726, -107.8514465
East Quogue
3 / 20136
40.851465000000005, -72.57792197443786
Indian River Shores
4 / 20136
27.7166964, -80.3842173
lake elsinore
5 / 20136
33.6680772, -117.3272615
Jersey City
6 / 20136
40.7281575, -74.0776417
Pembroke
7 / 20136
45.8260909, -77.113541
South Salem
8 / 20136
39.3361754, -83.3065796
Hayesville
9 / 20136
41.2640523, -92.2491537
Quartzite
10 / 20136
39.0211169, -98.0947728
Virginia City
11 / 20136
45.293796, -111.946123
Rosemead
12 / 20136
34.0676169, -118.0879763
Long Beach
13 / 20136
33.7690164, -118.191604
Thousand Oaks
14 / 20136
34.17142715, -118.91058772974445
Gulfport
15 / 20136
30.3674198, -89.0928155
Cathedral City
16 / 20136
33.822998049999995, -116.4668702427116
Los Banos
17 / 20136
37.0592253, -120.8505342
Starkville
18 / 20136
33.4503998, -88.8183872
Belleville
19 / 20136
48.8176714, 6.0982683
Escondido.
20 / 20136
33.1216751, -117.0814849
Maidenhead
21 / 20136
51.5237638, -0.718806
Albuquerque
22 / 20136
35.0841034, -106.650985

<ipython-input-426-e8d1709e9bfb>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["latitude"] = np.where(isCity, 'nan', noMADAR["latitude"])
<ipython-input-426-e8d1709e9bfb>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["longitude"] = np.where(isCity, 'nan', noMADAR["longitude"])


79 /20136
Anaheim
80 / 20136
33.8347516, -117.911732
York Harbor
81 / 20136
43.1367551, -70.6456078
Port Orchard
82 / 20136
47.5315625, -122.6384056
Benton
83 / 20136
45.7162129, -94.0481042
Golden
84 / 20136
39.743310699999995, -105.21250036061394
Modesto
85 / 20136
37.6390972, -120.9968782
Gloucester
86 / 20136
51.8653705, -2.2458192
Bend
87 / 20136
44.0581728, -121.3153096
Great Meadows
88 / 20136
40.8709319, -74.9121142
Denver
89 / 20136
39.7392364, -104.9848623
Nogales
90 / 20136
38.5865813, -6.7491704
OCEAN CITY
91 / 20136
39.2776156, -74.5746001
Fredericksburg
92 / 20136
38.3031837, -77.4605399
Eureka
93 / 20136
40.8020712, -124.1636729
Princeton
94 / 20136
40.3492744, -74.6592958
Pacifica
95 / 20136
37.6138253, -122.4869194
Anamoose
96 / 20136
47.8826593, -100.240959
Colorado Springs
97 / 20136
38.8339578, -104.8253485
Garden Valley
98 / 20136
44.089894, -115.9520638
ranchos de taos
99 / 20136
36.3586972, -105.6093439
Juno Beach
100 / 20136
49.3364567, -0.4512533175742328
Rockl

...

6870 / 20136
42.4620142, -83.1035688
Northfield (Canada)
6871 / 20136
44.4766047, -65.1587534
Munford
6872 / 20136
35.44925, -89.8150833
Pearl Harbor
6873 / 20136
21.35577485, -157.99912976013587
Dearborn Heights
6874 / 20136
42.3369816, -83.2732627
Fort Bragg (close to)
6875 /20136
Lake
6876 / 20136
39.0505411, -122.7776556
Khammam (India)
6877 / 20136
17.5, 80.333333
Island Læsø (Laesoe)(Denmark)
6878 / 20136
57.271898, 10.992758220967776
Detroit (northwest area)
6879 / 20136
42.3672046, -83.15042821099715
Edinburgh (UK/Scotland)
6880 / 20136
55.9533456, -3.1883749
Alto
6881 / 20136
44.1089458, 8.003074
Mantorville
6882 / 20136
44.0691307, -92.7557456
Medford/Weed
6883 /20136
Blyton
6884 / 20136
40.5642068, -90.2765127
Macedonia
6885 / 20136
41.6171214, 21.7168387
Newfoundland
6886 / 20136
49.312, -56.474908221771685
Williamson
6887 / 20136
30.6580927, -97.6041649
Vadnais Heights/White Bear Township
6888 /20136
Stoney Creek
6889 / 20136
43.2167722, -79.7567508
Mount Clemens
6890 / 20

7033 / 20136
41.3345799, -94.0143859
North Muskegon
7034 / 20136
43.256125, -86.26756
Rock Springs
7035 / 20136
41.5869225, -109.2047867
Moscow Mills
7036 / 20136
38.9478267, -90.9181916
Nixa
7037 / 20136
37.0439358, -93.3098534
Assateague Island
7038 / 20136
38.08718645, -75.20999471221393
Lusaka (Zambia)
7039 / 20136
-15.4164488, 28.2821535
Epsom
7040 / 20136
51.3326098, -0.2678212
Custer
7041 / 20136
38.1072962, -105.3543375
St. Croix (Canada)
7042 / 20136
44.9619372, -64.0337916
Trigg County (Lake Barkley)
7043 / 20136
36.8018323, -87.98869421811024
Rushford
7044 / 20136
43.8082606, -91.7531265
Alicante (Spain)
7045 / 20136
14.0616729, 121.316061
Bemow Piskie (Poland)
7046 /20136
La Habra
7047 / 20136
33.9316066, -117.9454867
Deltona
7048 / 20136
28.9005446, -81.2636738
Buzzards Bay
7049 / 20136
41.745383, -70.618087
Severna Park
7050 / 20136
39.070388, -76.5452409
Deepwater
7051 / 20136
38.26058, -93.7729902
Birmingham (outside Hueytown)
7052 /20136
Santa rosa
7053 / 20136
14.1501

7194 /20136
Karnata Gadak (India)
7195 /20136
Lapeer
7196 / 20136
43.0904764, -83.2333705
Port Barre
7197 / 20136
30.5601953, -91.9540055
Maxton
7198 / 20136
34.7351615, -79.3489317
Donora
7199 / 20136
40.1734049, -79.8575467
Pocono Lake
7200 / 20136
41.1053652, -75.4760193
Redwater (Canada)
7201 / 20136
53.9527743, -113.1124871
Lantana
7202 / 20136
33.094027600000004, -97.12449696030463
Rutledge
7203 / 20136
46.2595839, -92.8664568
Turnersville
7204 / 20136
39.766826, -75.06300404150056
New York City (Queens)(Flushing Meadows)
7205 /20136
Vilnius (Lithuania)
7206 / 20136
54.6870458, 25.2829111
Ruwais (U.A.E.)
7207 /20136
Novi
7208 / 20136
42.48059, -83.4754913
Croton
7209 / 20136
40.5109368, -74.9301674
Rabbit Valley
7210 / 20136
44.1704157, -120.0692368
Cherokee
7211 / 20136
31.8390826, -95.1791849
Leota
7212 / 20136
48.3150709, 17.9927535
Cimarron
7213 / 20136
37.8066881, -100.3482003
Ballinger
7214 / 20136
31.7411047, -99.9530698
Allen Park
7215 / 20136
42.2595071, -83.2107671
Lock

7364 / 20136
46.6540065, -120.5302727
Oneco
7365 / 20136
42.4874124, -89.6645362
Thompson (Canada)
7366 / 20136
55.743347, -97.863464
Sanra Rosa
7367 / 20136
-27.8706324, -54.4775697
Stevenson Ranch
7368 / 20136
34.3881783, -118.5853957
Wanica (Surinam)
7369 / 20136
5.75, -55.25
North River (Canada)
7370 / 20136
53.5338302, -60.14179926167898
Hanging Rock
7371 / 20136
38.56008, -82.7209975
Belgrade (Serbia)
7372 / 20136
44.8178131, 20.4568974
Morden (Surrey) (UK/England)
7373 /20136
Bridgeview
7374 / 20136
41.7500323, -87.8042216
Chiba City (Japan)
7375 / 20136
35.609985, 140.118126
Holiday Island
7376 / 20136
44.3039368, -75.9774502
Uniontown/Kent
7377 /20136
Galena Park
7378 / 20136
29.7335616, -95.2302123
Cottage Grove & Madison
7379 / 20136
34.737702799999994, -86.7873684
Bermuda/Florida coast (between)
7380 /20136
New York City (upper West Side)
7381 / 20136
40.7870455, -73.9754163
Les Cayes (Haiti)
7382 / 20136
18.1944428, -73.7511601
Ropar (Punjab)(India)
7383 / 20136
30.9714438

7525 / 20136
38.2207112, -90.3959504
Fountain Valley
7526 / 20136
33.7038145, -117.9627349
Tamaqua
7527 / 20136
40.7974296, -75.9696186
Bowen Island (Canada)
7528 / 20136
46.3184452, -84.0641908
Swanville
7529 / 20136
45.91479914999999, -94.63837562453601
Nederland
7530 / 20136
52.5001698, 5.7480821
Basehor
7531 / 20136
39.1416692, -94.9385763
Porter township
7532 / 20136
41.83116455, -85.81833479968817
Parachute
7533 / 20136
39.4536891, -108.0529619
Hiwassee
7534 / 20136
36.9687335, -80.7136789
Illogan (UK/England; southwest region)
7535 /20136
Lindenwold
7536 / 20136
39.81811999999999, -74.97996221777835
South Dennis
7537 / 20136
41.6895552, -70.1564077
Midland (Canada)
7538 / 20136
44.750147, -79.885712
Grandville
7539 / 20136
48.5956451, 4.2319159
Waimea Canyon
7540 / 20136
21.992453, -159.674102
Mananlapan
7541 /20136
Mt. Summit
7542 / 20136
40.0044908, -85.3846928
Elk Mountain
7543 / 20136
40.1619295, -106.1283562
Londonderry
7544 / 20136
54.9978678, -7.3213056
Board Camp
7545 / 

In [439]:
noMADAR["latitude"].fillna(0.000, inplace=True)
noMADAR["longitude"].fillna(0.000, inplace=True)

In [448]:
noMADAR["latitude"] = noMADAR["latitude"].astype(float)
noMADAR["longitude"] = noMADAR["longitude"].astype(float)

<ipython-input-448-025add79bc67>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["latitude"] = noMADAR["latitude"].astype(float)
<ipython-input-448-025add79bc67>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  noMADAR["longitude"] = noMADAR["longitude"].astype(float)


In [449]:
noMADAR.dtypes

datetime     datetime64[ns]
city                 object
country              object
shape                object
comment              object
latitude            float64
longitude           float64
dtype: object